In [15]:
import pandas as pd
import numpy as np
import re
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from scipy.stats import pearsonr

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)

print(filtered_survival_df.shape)
filtered_survival_df.head()

(66, 16)


,sample_name,survival_time,age_at_diagnosis,race_american indian or alaska native,race_asian,race_black or african american,race_native hawaiian or other pacific islander,race_not reported,race_white,ethnicity_hispanic or latino,ethnicity_not hispanic or latino,ethnicity_not reported,figo_chr_figo_stage_1,figo_chr_figo_stage_2,figo_chr_figo_stage_3,figo_chr_figo_stage_4
0,TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975.0,0,0,0,0,0,1,0,0,1,1,0,0,0
1,TCGA-C5-A8YT-01A-11R-A37O-07,633,13253.0,0,0,0,0,0,1,0,1,0,1,0,0,0
2,TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727.0,0,0,0,0,0,1,0,0,1,1,0,0,0
3,TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444.0,0,0,0,0,0,1,0,1,0,1,0,0,0
4,TCGA-DS-A7WF-01A-11R-A352-07,492,15319.0,0,0,0,0,1,0,1,0,0,1,0,0,0


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


,sample_name,PGF,TIMP4,C1QTNF6,TNC,PRL,OGN,C1QL3,FGB,NDNF,...,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
0,TCGA-C5-A1BF-01B-11R-A13Y-07,10.800637,6.228003,11.669331,13.002928,5.063964,4.869744,5.063964,8.834522,6.410767,...,9.013453,8.190325,9.503647,14.077995,6.569726,7.315604,4.602649,12.062300,5.649441,16.558407
1,TCGA-C5-A8YT-01A-11R-A37O-07,7.830611,5.733875,12.445548,13.765468,5.455125,13.049104,5.146455,5.074289,10.569544,...,9.453187,6.398956,12.288955,13.396332,10.228758,8.542025,4.602649,11.765396,5.318924,13.556322
2,TCGA-C5-A1BE-01B-11R-A13Y-07,10.642039,5.348449,8.945220,13.419225,4.602649,5.867905,5.646251,4.602649,5.673617,...,11.696884,6.384980,9.774029,15.381944,6.205261,7.163053,4.957257,10.113788,5.213815,15.564969
3,TCGA-C5-A8XH-01A-11R-A37O-07,9.633752,5.908552,11.672191,13.863766,4.602649,5.339887,5.702581,5.389600,5.634507,...,11.148165,7.526140,8.561116,14.404419,6.462928,6.102570,4.602649,9.104547,5.088257,15.197350
4,TCGA-DS-A7WF-01A-11R-A352-07,13.292479,5.620704,12.867887,16.646519,4.602649,10.377267,6.177498,4.602649,5.198452,...,10.809104,7.877841,6.615459,13.894278,7.058159,10.809104,4.602649,15.271686,6.519692,16.820793


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
joined_df.head()

(66, 1023)


,survival_time,age_at_diagnosis,race_american indian or alaska native,race_asian,race_black or african american,race_native hawaiian or other pacific islander,race_not reported,race_white,ethnicity_hispanic or latino,ethnicity_not hispanic or latino,...,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
sample_name,,,,,,,,,,,,,,,,,,,,,
TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975.0,0,0,0,0,0,1,0,0,...,9.013453,8.190325,9.503647,14.077995,6.569726,7.315604,4.602649,12.062300,5.649441,16.558407
TCGA-C5-A8YT-01A-11R-A37O-07,633,13253.0,0,0,0,0,0,1,0,1,...,9.453187,6.398956,12.288955,13.396332,10.228758,8.542025,4.602649,11.765396,5.318924,13.556322
TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727.0,0,0,0,0,0,1,0,0,...,11.696884,6.384980,9.774029,15.381944,6.205261,7.163053,4.957257,10.113788,5.213815,15.564969
TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444.0,0,0,0,0,0,1,0,1,...,11.148165,7.526140,8.561116,14.404419,6.462928,6.102570,4.602649,9.104547,5.088257,15.197350
TCGA-DS-A7WF-01A-11R-A352-07,492,15319.0,0,0,0,0,1,0,1,0,...,10.809104,7.877841,6.615459,13.894278,7.058159,10.809104,4.602649,15.271686,6.519692,16.820793


# PLSR

In [10]:
rand.seed(123)
shuffled_df = joined_df.sample(frac=1, random_state=rand)
X_df = shuffled_df.iloc[:, 1:]
# Y must be a matrix
Y_df = shuffled_df.iloc[:, [1]]

In [49]:
plsr_model = PLSRegression(scale=False)
plsr_pipeline = make_pipeline(
    ColumnTransformer([
        ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
    ], remainder="passthrough"),
    plsr_model
)
ttr = TransformedTargetRegressor(regressor=plsr_pipeline, transformer=StandardScaler())
# h_params = {"plsregression__n_components": range(2, 20)}
h_params = {"regressor__plsregression__n_components": range(2, 20)}
cv_grid_search = GridSearchCV(ttr, h_params, scoring="neg_mean_absolute_error", cv=KFold(5), n_jobs=-1, verbose=1)
cv_grid_search.fit(X_df, Y_df)
# cv_grid_search.best_params_
pd.DataFrame(cv_grid_search.cv_results_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  59 out of  90 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    0.6s finished


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__plsregression__n_components,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.025907,0.002101,0.012369,0.002668,2,{'regressor__plsregression__n_components': 2},-3813.573922,-4336.342954,-4520.643949,-4689.211197,-3067.550209,-4085.464446,587.691282,1
1,0.027263,0.004443,0.012559,0.002913,3,{'regressor__plsregression__n_components': 3},-3389.591458,-4792.052877,-5001.678875,-5184.118440,-3396.404484,-4352.769227,793.415441,2
2,0.033064,0.006841,0.017260,0.004197,4,{'regressor__plsregression__n_components': 4},-4006.563084,-4601.053834,-4978.146586,-5073.821851,-3547.575325,-4441.432136,583.360837,4
3,0.045148,0.001634,0.020057,0.000689,5,{'regressor__plsregression__n_components': 5},-4236.577771,-4564.567250,-4954.949122,-5058.842027,-3303.272220,-4423.641678,631.798659,3
4,0.049202,0.002215,0.018683,0.001178,6,{'regressor__plsregression__n_components': 6},-4435.634320,-4645.161955,-4763.272072,-5092.664243,-3346.882788,-4456.723076,594.321944,17
5,0.047882,0.003715,0.014937,0.001781,7,{'regressor__plsregression__n_components': 7},-4346.909544,-4690.308224,-4784.750249,-5037.504426,-3422.502376,-4456.394964,562.341845,16
6,0.040052,0.003825,0.016133,0.002318,8,{'regressor__plsregression__n_components': 8},-4352.150809,-4693.346700,-4802.210523,-5097.407096,-3350.836849,-4459.190395,603.277315,18
7,0.034256,0.005757,0.015071,0.003352,9,{'regressor__plsregression__n_components': 9},-4361.277265,-4685.534754,-4779.833639,-5081.332550,-3363.634010,-4454.322444,591.746062,15
8,0.036165,0.004672,0.017845,0.002769,10,{'regressor__plsregression__n_components': 10},-4342.119555,-4683.875218,-4754.852263,-5093.219942,-3369.605497,-4448.734495,589.957097,7
9,0.047257,0.001788,0.020943,0.002204,11,{'regressor__plsregression__n_components': 11},-4341.150256,-4685.845198,-4749.622822,-5086.925758,-3372.421006,-4447.193008,587.206365,5


In [50]:
best_model = cv_grid_search.best_estimator_.regressor_["plsregression"]
cv_score = cross_val_score(cv_grid_search.best_estimator_, X_df, Y_df, cv=KFold(5), scoring="neg_mean_absolute_error", n_jobs=-1)
print(cv_score.mean())
coef_df = pd.DataFrame({"var": shuffled_df.columns[1:], "coef": best_model.coef_[:, 0]}).assign(abs_coef = lambda x: np.abs(x.coef))

-4085.464446158928


In [52]:
coef_df.sort_values("abs_coef", ascending=False)[:20]

,var,coef,abs_coef
0,age_at_diagnosis,0.042939,0.042939
59,CSTA,-0.016041,0.016041
620,COL17A1,-0.015262,0.015262
704,FLG2,0.015144,0.015144
886,LGALS14,-0.015127,0.015127
779,SERPINA12,0.015073,0.015073
952,S100A12,0.014519,0.014519
882,WNT5B,-0.014365,0.014365
844,CSHL1,-0.014294,0.014294
267,INS,0.013728,0.013728
